In [1]:
# useing this variable for saving models and summaries
exp_name = "RCNN_Wmultitask_8_opp"

## Importing Libraries

In [4]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import models.CNN_lstm_model as CNN_lstm_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [7]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [8]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [9]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [10]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [11]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

pooling_stride = [2, 2, 2]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
activation = tf.nn.relu

lstm_n_hidden = 128
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)

pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [12]:
pos_weight.shape

(19,)

In [13]:
pos_weight

array([  0.40793684,  51.83632019,  50.54312354,  53.39606396,
        53.49661121,  43.82919412,  45.64978903,  64.85852569,
        69.47649402,  84.21001927,  98.26823793,  89.34525026,
        99.39500568,  75.77777778,  76.17975567,  46.17226667,
        13.60019809,  65.60240964,  20.4471387 ])

In [14]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [15]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/' + exp_name 

In [16]:
model = CNN_lstm_model.Model(n_variables = n_variables,
                        n_classes = n_classes)

In [17]:
model.build_cnn_lstm_net(pooling_stride = pooling_stride,
                         weights_shape = weights_shape,
                         bias_shape = bias_shape,
                         activation = activation,
                         lstm_n_hidden = lstm_n_hidden,
                         n_classes = n_classes,
                         learning_rate = learning_rate,
                         pos_weight = pos_weight,
                         trainable_conv = True,
                         trainable_last = True,
                         variables_weights = variables_weights)

In [18]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [19]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.056 vs. 0.058
F1 = 0.098 vs. 0.083
Accuracy = 0.012 vs. 0.056
CHANGE DETECTION
AUC = 0.037 vs. 0.040
F1 = 0.079 vs. 0.081
Precision = 0.041 vs. 0.042
Recall = 0.998 vs. 0.884
Validation
ACTIVITY RECOGNITION
AUC = 0.058 vs. 0.056
F1 = 0.085 vs. 0.075
Accuracy = 0.018 vs. 0.056
CHANGE DETECTION
AUC = 0.025 vs. 0.027
F1 = 0.055 vs. 0.052
Precision = 0.031 vs. 0.027
Recall = 0.235 vs. 0.809
Minibatch Loss= 1.712918
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.673 vs. 0.058
F1 = 0.658 vs. 0.083
Accuracy = 0.690 vs. 0.056
CHANGE DETECTION
AUC = 0.163 vs. 0.040
F1 = 0.243 vs. 0.082
Precision = 0.180 vs. 0.043
Recall = 0.378 vs. 0.993
Validation
ACTIVITY RECOGNITION
AUC = 0.476 vs. 0.057
F1 = 0.506 vs. 0.076
Accuracy = 0.674 vs. 0.056
CHANGE DETECTION
AUC = 0.102 vs. 0.027
F1 = 0.177 vs. 0.052
Precision = 0.107 vs. 0.027
Recall = 0.505 vs. 0.688
Minibatch Loss= 0.247715
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.822 vs. 0.059


In [24]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model/RCNN_Wmultitask_8_opp/RCNN_Wmultitask_8_opp-20000'

In [ ]:
#load the model
#saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
#saver.restore(sess, '../model/CNN7002_Gleam/CNN7002_Gleam-10000')


In [25]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.631 vs. 0.057
F1 = 0.646 vs. 0.076
Accuracy = 0.867 vs. 0.056
CHANGE DETECTION
AUC = 0.144 vs. 0.027
F1 = 0.247 vs. 0.054
Precision = 0.176 vs. 0.028
Recall = 0.418 vs. 0.485


In [26]:
out = nn_eval.deploy (model, sess, test_data, test_gt)

In [27]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(probability_of_pos, axis = 2)

In [28]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})

In [21]:
1

1

In [22]:
2

2